## Switching Phase Diagram with varying bias field ##
### for both AP->P and P->AP polarity ###
**Note: Data for each bias field and polarity will be written in a separate HDF5 file**

In [ ]:
# Experimental Topology
# lockin AO 2 -> Analog Attenuator Vdd
# lockin AO 3 -> Analog Attenuator Vc (Control Voltages)
# Keithley Output -> Voltage divider with 1 MOhm, DAQmx AI1
# AWG Sync Marker Out -> DAQmx PFI0
# AWG Samp. Marker Out -> PSPL Trigger

In [1]:
cd ..

C:\Users\qlab\Documents\GitHub\pycontrol


In [2]:
from pycontrol.filters.io import WriteToHDF5
import numpy as np
import itertools
import time
import matplotlib.pyplot as plt
import analysis.switching as sw
from adapt import refine
from analysis.h5shell import h5shell
from tqdm import tqdm_notebook

In [3]:
from experiments.Pulse_Switching_CSHE import SwitchingExperiment

In [4]:
polarities = [-1,1]
# fields = np.linspace(0,160,9)*1e-4
fields = np.linspace(81,71,2)*1e-4

coarse_ts = 1e-9*np.linspace(0.1, 10, 5) # List of durations
coarse_vs = np.linspace(0.3, 1.0, 5)

In [5]:
exp = SwitchingExperiment()
exp.sample = "CSHE5-C2R3"
exp.iteration = 2
exp.measure_current = 3e-6
exp.reset_amplitude = 0.7
exp.reset_duration  = 5.0e-9
exp.settle_delay    = 200e-6
exp.pspl_atten      = 5

folder = "data\CSHE-Switching\CSHE-Die5-C2R3\Phase_Diagram_FieldVary_2016-07-27\\"
ftracks = []
figs = []
points    = [coarse_ts, coarse_vs]
points    = list(itertools.product(*points))
exp.init_streams()
main_sweep = exp.add_unstructured_sweep([exp.pulse_duration, exp.pulse_voltage], points)

In [6]:
for field in tqdm_notebook(fields, total=len(fields)):
    for polarity in polarities:
        exp.comment = "Phase Diagram -  Varying Field - Interations = 1 - Settle delay = 200us - Reset 0.7V 5ns"
        exp.comment = exp.comment + " - Field = " + str(field) + "T"
        polarity_str = " AP to P" if polarity==1 else " P to AP"
        exp.comment = exp.comment + polarity_str
        exp.polarity = polarity
        exp.field.value = field
        
        print("Start measurement with field = {}T and polarity = {}".format(field,polarity_str))
        
        wr = WriteToHDF5(folder + "CSHE5-C2R3_FieldVary_2016-07-27.h5")
        edges = [(exp.daq_buffer, wr.data)]
        exp.set_graph(edges)
        
        # Keep track of filenames
        track = {}
        track['filename'] = wr.filename
        track['polarity'] = exp.polarity
        track['polarity_str'] = exp.polarity_str
        track['field'] = field
        track['comment'] = exp.comment
        ftracks.append(track)
        
        points    = [coarse_ts, coarse_vs]
        points    = list(itertools.product(*points))
        main_sweep.update_values(points)
        exp.init_instruments()
        t1 = time.time()
        for i in tqdm_notebook(range(exp.iteration), total=exp.iteration):
            exp.reset()
#             exp.init_progressbar(1, notebook=True)
            exp.run_sweeps()
            points, mean = sw.load_switching_data(wr.filename, threshold=0.36)
            new_points = refine.refine_scalar_field(points, mean, all_points=False,
                                        criterion="integral", threshold = "one_sigma")
            if new_points is None:
                print("No more points can be added.")
                break
            main_sweep.update_values(new_points)
            exp.progressbar.close()
        t2 = time.time()
        print("Done one measurement.")
        print("Total points: {}".format(len(points)))
        print("Total elapsed time: {} min.".format((t2-t1)/60))
        print("Data file: %s" %wr.filename)
        fig_title = exp.sample  +polarity_str + " at field=" + str(exp.field.value) + "T"
        figs.append(sw.phase_diagram_mesh(points, mean, title=fig_title))
        print("=========")
        
exp.shutdown_instruments()
print("Finished measurement. Shutted down.")

Start measurement with field = 0.0081T and polarity =  P to AP
10 new points added.
14 new points added.
Done one measurement.
Total points: 35
Total elapsed time: 0.5529570500055949 min.
Data file: data\CSHE-Switching\CSHE-Die5-C2R3\test\CSHE5-C2R3_FieldVary_2016-07-27-0005.h5
Start measurement with field = 0.0081T and polarity =  AP to P
8 new points added.
21 new points added.
Done one measurement.
Total points: 33
Total elapsed time: 0.5470046718915303 min.
Data file: data\CSHE-Switching\CSHE-Die5-C2R3\test\CSHE5-C2R3_FieldVary_2016-07-27-0006.h5
Start measurement with field = 0.0071T and polarity =  P to AP
10 new points added.
16 new points added.
Done one measurement.
Total points: 35
Total elapsed time: 0.6823741436004639 min.
Data file: data\CSHE-Switching\CSHE-Die5-C2R3\test\CSHE5-C2R3_FieldVary_2016-07-27-0007.h5
Start measurement with field = 0.0071T and polarity =  AP to P
8 new points added.
21 new points added.
Done one measurement.
Total points: 33
Total elapsed time: 0

In [7]:
plt.show()

In [8]:
import importlib
importlib.reload(sw)

<module 'analysis.switching' from 'C:\\Users\\qlab\\Documents\\GitHub\\pycontrol\\analysis\\switching.py'>